# 광고영상 프레임들 훈련용 / 검증용 셋으로 편입 

In [ ]:
import os 
import sys
import shutil 

sys.path.append('/Users/kibeomkim/Desktop/영상프레임/')

video_list = os.listdir('/Users/kibeomkim/Desktop/영상프레임')
video_list.remove('.DS_Store')
# 훈련용 비디오 리스트 
train_video_list = video_list[:25]
# 테스트용 비디오 리스트 
test_video_list = video_list[25:]

for video in train_video_list : 
    for frame in os.listdir(f'/Users/kibeomkim/Desktop/영상프레임/{video}') : # 훈련용 비디오 프레임 하나하나에 대해서 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/영상프레임/{video}/{frame}', f'/Users/kibeomkim/Desktop/openlogo_video/train/samsung/{frame}')

for video in test_video_list : 
    for frame in os.listdir(f'/Users/kibeomkim/Desktop/영상프레임/{video}') : # 테스트 비디오 프레임 하나하나에 대해서 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/영상프레임/{video}/{frame}', f'/Users/kibeomkim/Desktop/openlogo_video/test/samsung/{frame}')

# Xception 모델에 갤럭시 S21 광고 영상 프레임들 넣기 

In [17]:
import keras 
from keras.applications import xception
from keras import layers
from keras import preprocessing

base_model = keras.applications.xception.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

norm_layer = keras.layers.BatchNormalization()
x = norm_layer(inputs)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_29 (Bat  (None, 150, 150, 3)      12        
 chNormalization)                                                
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_5   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_5 (Dropout)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 1)                 2049

In [18]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.6,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_video/train', 
    target_size=(150, 150),
    batch_size=100, 
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_video/test',
    target_size=(150,150),
    batch_size=100,
    class_mode='binary'
)

Found 40783 images belonging to 2 classes.
Found 17106 images belonging to 2 classes.


In [19]:
# 가장 기본적인 3가지 콜백 
callbacks_list = [

    #과대적합 발생 시 조기 스톱 콜백
    keras.callbacks.EarlyStopping(
        monitor='binary_accuracy', # 모델 검증 손실 모니터링
        patience= 10), # 20에폭동안 과대적합 관찰
    
    # 최고의 가중치 자동 저장 콜백
    keras.callbacks.ModelCheckpoint(
        '/Users/kibeomkim/Desktop/models_saved/my_model.h5', 
        monitor = 'val_loss', 
        save_best_only = True), 

    # 학습률 자동 조정 콜백
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=5
    ),

    # 텐서보드 
    keras.callbacks.TensorBoard(
        log_dir = '/Users/kibeomkim/Desktop/my_log_dir',
        histogram_freq = 1, # 1에포크마다 층 출력의 히스토그램 기록
        embeddings_freq = 1
    )
]

In [20]:
import keras.optimizers

# 모델 컴파일 
model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(), 
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

# 모델 훈련 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 407, 
    epochs=5,
    validation_data = validation_generator, 
    validation_steps=171, 
    callbacks=callbacks_list 
)

Epoch 1/5


2022-02-24 12:46:25.576218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


407/407 [==============================] - ETA: 0s - loss: 0.6917 - binary_accuracy: 0.5361

2022-02-24 13:00:11.281170: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


407/407 [==============================] - 951s 2s/step - loss: 0.6917 - binary_accuracy: 0.5361 - val_loss: 0.6967 - val_binary_accuracy: 0.5141 - lr: 0.0010
Epoch 2/5
407/407 [==============================] - 882s 2s/step - loss: 0.6903 - binary_accuracy: 0.5397 - val_loss: 0.6951 - val_binary_accuracy: 0.5140 - lr: 0.0010
Epoch 3/5
407/407 [==============================] - 888s 2s/step - loss: 0.6905 - binary_accuracy: 0.5391 - val_loss: 0.6937 - val_binary_accuracy: 0.5141 - lr: 0.0010
Epoch 4/5
407/407 [==============================] - ETA: 0s - loss: 0.6903 - binary_accuracy: 0.5413

KeyboardInterrupt: 

In [58]:
model.save('/Users/kibeomkim/Desktop/models_saved/video_trained_model.h5') # 모델 저장 

In [64]:
# 미세조정 
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(learning_rate=1e-5), 
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 407, 
    epochs=1,
    validation_data = validation_generator, 
    validation_steps=171, 
    callbacks=callbacks_list 
)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_34 (Bat  (None, 150, 150, 3)      12        
 chNormalization)                                                
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_6   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_6 (Dropout)         (None, 2048)              0         
                                                                 
 dense_6 (Dense)             (None, 2)                 4098

2022-02-24 10:57:01.517156: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


407/407 [==============================] - ETA: 0s - loss: 1.0450 - binary_accuracy: 0.9047

2022-02-24 11:31:48.445260: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


407/407 [==============================] - 2218s 5s/step - loss: 1.0450 - binary_accuracy: 0.9047 - val_loss: 1.1603 - val_binary_accuracy: 0.8716 - lr: 1.0000e-05


In [65]:
model.save('/Users/kibeomkim/Desktop/models_saved/fine_tuned_video_trained_model.h5') # 모델 저장 